# Propósito
Revisar los distintos usos que se le pueden dar a los hashes en Python.

# ¿Qué es el hashing en Python?
Consiste en transformar una serie de datos de entrada en una cadena de bytes de tamaño fijo. Cada entrada genera una cadena única, por lo que los hashes permiten reproducibilidad y un almacenamiento más seguro de los datos.

En python, la generación de hashes además dependerá de una semilla, por lo que si no se fija, cada instancia dará como resultado hashes distintos para la misma cadena de texto.

Los distintos algoritmos de hasheo deben mostrar una serie de propiedades:
- Deterministas: Ante el mismo input, deben generar el mismo hash
- Computacionalmente eficiente, de rápido acceso
- Únicos: Cada input debe generar un hash único e irrepetible.



In [ ]:
import hashlib # Importamos la librería hashlib, nos permite generar hashes de diferentes algoritmos.
import hmac
import os
import json
import time
from typing import Dict, Any, Tuple, Optional
text= "Hello World!"
hash_object =hashlib.md5(text.encode())
print(hash_object.hexdigest()) # Convertimos el hash a hexadecimal.

ed076287532e86365e841e92bfc50d8c


## Casos de uso: 
### Hashing archivo
Debemos leer el archivo en chunks o bloques, que serán convertidos y actualizarán el hash. Una vez que hayamos leído el archivo completo, podemos usar digest() para producir el hash final.

In [8]:
import hashlib
import base64

def hash_file_sha256(filepath):
    """
    Computes the SHA-256 hash of the specified file.

    :param filepath: Path to the file to be hashed
    :return: Hexadecimal SHA-256 hash string
    """
    sha256_hash = hashlib.sha256()
    try:
        with open(filepath, "rb") as f:
            # Leemos y actualizamos el hash en bloques de 4096 bytes.
            for byte_block in iter(lambda: f.read(4096), b""):
                sha256_hash.update(byte_block) # Actualizamos el hash con cada bloque.
        return sha256_hash.digest() # Devolvemos el hash en formato binario.
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [9]:
# Prueba
hash = hash_file_sha256("C:/Users/yanni/Documents/Codigo/Proyectos/python_hashes.ipynb")
print(hash)
print(hash.hex()) 
print(base64.b64encode(hash).decode("utf-8")) 

b'w6\\\xda(\xd6\xa8C\xdc\xfa\x14\xb7o\x92;;\x15\xa1Liw\x0e\x02l\xd2\x05jp\x07\x17l`'
77365cda28d6a843dcfa14b76f923b3b15a14c69770e026cd2056a7007176c60
dzZc2ijWqEPc+hS3b5I7OxWhTGl3DgJs0gVqcAcXbGA=


### Almacenamiento de contraseñas:
Hacemos uso de otra técnica llamada salting para hashear las contraseñas.

El salting nos permite añadir una capa de seguridad al proceso de hasheo de las contraseñas, añadiendo un valor único a cada contraseña.

Podemos identificar dos procesos:
- Creación de contraseña
- Verificación de contraseña

In [10]:
# Source: https://stackoverflow.com/questions/9594125/salt-and-hash-a-password-in-python

from typing import Tuple
import os
import hashlib
import hmac
import base64

def hash_new_password(password: str) -> Tuple[bytes, bytes]:
    """
    Hash the provided password with a randomly-generated salt and return the
    salt and hash to store in the database.
    """
    salt = os.urandom(16)
    pw_hash = hashlib.pbkdf2_hmac('sha256', password.encode(), salt, 1000000)
    return salt, pw_hash

def is_correct_password(salt: bytes, pw_hash: bytes, password: str) -> bool:
    """
    Given a previously-stored salt and hash, and a password provided by a user
    trying to log in, check whether the password is correct.
    """
    return hmac.compare_digest(
        pw_hash,
        hashlib.pbkdf2_hmac('sha256', password.encode(), salt, 1000000)
    )

# Example usage:
salt, pw_hash = hash_new_password('correct horse battery staple')
print(base64.b64encode(salt))
print(base64.b64encode(pw_hash))
print(salt.hex())
print(pw_hash.hex())
assert is_correct_password(salt, pw_hash, 'correct horse battery staple')
assert not is_correct_password(salt, pw_hash, 'Tr0ub4dor&3')
assert not is_correct_password(salt, pw_hash, 'rosebud')

b'XM2HZuFLGdltrzGt2oQTvQ=='
b'WeBMH1WAF9q8efTUnnsdhXC69h5IEL5joGEXtFntCFg='
5ccd8766e14b19d96daf31adda8413bd
59e04c1f558017dabc79f4d49e7b1d8570baf61e4810be63a06117b459ed0858


Fuentes:
- https://fintechpython.pages.oit.duke.edu/jupyternotebooks/2-Advanced%20Python/54-Cryptography.html